# Generate API Coverage Documentation

This notebook generates API coverage documentation from instrumentation
files and a pandas documentation reference, `DocumentedPandasAPI.csv`
when running snowpandas (either individual notebooks or through pytest).

## Generating the Results
- Run pytest with `--generate_pandas_api_coverage` to generate a set of `record-[TIMESTAMP].csv` files
- Change the `INSTRUMENTATION_FILES` variable below to point to these files (if needed)
- Ensure that `PANDA_DOCUMENTATION_MAP` points to the csv file containing all known pandas API calls
- Run the notebook. The record files will be collected and
concatinated into a dataframe.

## Interpreting the Results

- "✅" means that the function was executed without error every time
- "🟡" means that the function was executed and at least one invocation resulted in a NotImplementedError
- "❌" means that the function was never executed, it is not instrumented, or it produced only errors

Not all APIs are instrumented, and improvements to `PandasAPICoverageGenerator`
will be needed to ensure this API documentation is up to date.

In [61]:
INSTRUMENTATION_FILES="../../record-*.csv"
PANDA_DOCUMENTATION_MAP="DocumentedPandasAPI.csv"

In [62]:
import glob
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [ ]:
# Collect a set of record files from instrumentation
# into a dataframe
filenames=glob.glob(INSTRUMENTATION_FILES)
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, low_memory=True))
big_frame = pd.concat(dfs, ignore_index=True)
df = big_frame[["class", "method", "params", "exception", "start", "stop"]]

In [64]:
# Collect the documentation map into a dataframe
doc_df = pd.read_csv(PANDA_DOCUMENTATION_MAP)

In [ ]:
df['class'].value_counts()

In [ ]:
# Map the snowpandas class names to the pandas class names
df = df[['class', 'method', 'exception']]
df['method'] = np.where(df['class'] == 'DatetimeProperties', "dt." + df['method'], df['method'])
df['class'] = np.where(df['class'] == 'DatetimeProperties', "Series", df['class'])
df['class'] = np.where(df['class'] == 'snowflake.snowpark.modin.pandas', "pandas", df['class'])
df['class'] = np.where(df['class'] == 'modin.pandas', "pandas", df['class'])
df['class'] = np.where(df['class'] == 'SnowparkPandasDataFrame', "DataFrame", df['class'])
df['class'] = np.where(df['class'] == 'SnowparkPandasSeries', "Series", df['class'])
df['method'] = np.where(df['class'] == 'StringMethods', "str." + df['method'], df['method'])
df['class'] = np.where(df['class'] == 'StringMethods', "Series", df['class'])
df['class'] = np.where(df['class'] == 'class', "other", df['class'])
df['class'].value_counts()

In [67]:
# Any call w/o an exception is "Supported"
df['coverage'] = df['exception'].fillna("Supported")
# Map Supported -> ✅
#     NotImplementedError -> ❌
#     All other errors -> ⚠
df['coverage'] = np.where(df['coverage'] == 'Supported', "✅",
                          np.where(df['coverage'] == 'NotImplementedError', '❌','⚠'))
supported_df = df[['class', 'method', 'coverage']]

In [68]:
doc_df.columns= doc_df.columns.str.lower()
result_df=supported_df.merge(doc_df, on=['class', 'method'], how="right")
result_df['group']=result_df['group'].fillna("pandas")
result_df = result_df.pivot_table(index=['class', 'group', 'method'], values=['milestone', 'coverage'], 
                                  aggfunc={'coverage':pd.Series.unique, 'milestone':pd.Series.max}, fill_value="")
result_df['coverage'] = result_df['coverage'].apply(lambda x: 
                               '🟡' if "✅" in x and "❌" in x
                               else '✅' if "✅" in x and "❌" not in x
                               else '❌')

result_df = result_df.round(4)

In [69]:
# Write out API documentation
with open('Supported_API.md', 'w') as fo:
    fo.write(result_df.reset_index().to_markdown(index=True, tablefmt="github"))
# Write out API documentation
with open('Supported_API.csv', 'w') as fo:
    fo.write(result_df.reset_index().to_csv(index=True))

In [ ]:
final_df = result_df
final_df = final_df.reset_index()
final_df = final_df[['coverage']]
final_df_count = final_df.value_counts()
#final_df = final_df_count.reset_index()
#final_df["count"] = final_df.loc[:,'proportion']
final_df_count

In [71]:
def report_perc(by=None):
    final_df = result_df
    final_df = final_df.reset_index()
    if by is None:
        final_df = final_df[['coverage']]
        final_df_count = final_df.value_counts(normalize=True)
    else:
        final_df = final_df[[by, 'coverage']].sort_values(by=by)
        final_df_count = final_df.groupby(by).value_counts(normalize=True)
    final_df = final_df_count.reset_index()
    final_df["count"] = final_df.loc[:,'proportion']
    final_df.round(2)
    if by is not None:
        final_df = final_df.pivot_table(index=[by], columns=['coverage'], values=["count"],
                                  aggfunc={'count':pd.Series.sum})
    else:
        final_df = final_df.pivot_table(columns=['coverage'], values=["count"],
                                  aggfunc={'count':pd.Series.sum})

    return (final_df * 100).round(0).astype("str") + "%"
  
def report_num(by=None):
    final_df = result_df
    final_df = final_df.reset_index()
    if by is None:
        final_df = final_df[['coverage']]
        final_df_count = final_df.value_counts()
    else:
        final_df = final_df[[by, 'coverage']].sort_values(by=by)
        final_df_count = final_df.groupby(by).value_counts()
    final_df = final_df_count.reset_index()
    final_df["count"] = final_df.loc[:,'count']
    final_df.round(2)
    if by is not None:
        final_df = final_df.pivot_table(index=[by], columns=['coverage'], values=["count"],
                                  aggfunc={'count':pd.Series.sum})
    else:
        final_df = final_df.pivot_table(columns=['coverage'], values=["count"],
                                  aggfunc={'count':pd.Series.sum})

    return final_df

In [72]:
report_perc()

coverage,✅,❌,🟡
count,31.0%,60.0%,9.0%


In [73]:
report_num()

coverage,✅,❌,🟡
count,333,652,93


In [76]:
print("Snowpandas Supported API Count")
report_perc("class")

Snowpandas Supported API Count


count               
coverage                     ✅       ❌      🟡
class                                        
CategoricalIndex         15.0%   85.0%   nan%
DataFrame                36.0%   47.0%  17.0%
DataFrameGroupBy         22.0%   46.0%  32.0%
DatetimeIndex            43.0%   57.0%   nan%
Expanding                 nan%  100.0%   nan%
ExponentialMovingWindow   nan%  100.0%   nan%
Index                    36.0%   64.0%   nan%
IntervalIndex            17.0%   83.0%   nan%
MultiIndex               71.0%   29.0%   nan%
PeriodIndex               nan%  100.0%   nan%
RangeIndex               75.0%   25.0%   nan%
Resampler                27.0%   60.0%  13.0%
Rolling                  41.0%   53.0%   6.0%
Series                   37.0%   57.0%   6.0%
SeriesGroupBy            24.0%   65.0%  11.0%
TimedeltaIndex            nan%  100.0%   nan%
Window                    nan%  100.0%   nan%
api                       nan%  100.0%   nan%
pandas                    nan%   99.0%   1.0%

In [77]:
report_num("class")

count             
coverage                     ✅      ❌     🟡
class                                      
CategoricalIndex           2.0   11.0   NaN
DataFrame                 85.0  109.0  40.0
DataFrameGroupBy          13.0   27.0  19.0
DatetimeIndex             20.0   27.0   NaN
Expanding                  NaN   17.0   NaN
ExponentialMovingWindow    NaN    6.0   NaN
Index                     30.0   54.0   NaN
IntervalIndex              3.0   15.0   NaN
MultiIndex                20.0    8.0   NaN
PeriodIndex                NaN   25.0   NaN
RangeIndex                 3.0    1.0   NaN
Resampler                  8.0   18.0   4.0
Rolling                    7.0    9.0   1.0
Series                   127.0  194.0  21.0
SeriesGroupBy             15.0   40.0   7.0
TimedeltaIndex             NaN   14.0   NaN
Window                     NaN    4.0   NaN
api                        NaN    3.0   NaN
pandas                     NaN   70.0   1.0